In [1]:
import pandas as pd

df = pd.read_csv("tmp/interventions.csv")
df.head()

,intervenant,parti,intervention
0,Yaël Braun-Pivet,REN,\nL'ordre du jour appelle la discussion du pro...
1,Danièle Obono,LFI,"\nNon, pas question !"
2,Louis Boyard,LFI,"\nNon, ça ne va pas se passer comme ça ! Vous ..."
3,Yaël Braun-Pivet,REN,"\nLa parole est à M. le ministre du travail, d..."
4,André Chassaigne,GDR,\nSuspension ! La suspension est de droit !


In [2]:
from transformers import pipeline

# Load the emotion classification pipeline
emotion_classifier = pipeline('text-classification', 
                              model='bhadresh-savani/distilbert-base-uncased-emotion', 
                              tokenizer='bhadresh-savani/distilbert-base-uncased-emotion')

# Classify some sample text
text = "I am very happy today!"
result = emotion_classifier(text)

# Print the predicted emotion and its score
print(result[0]['label'], result[0]['score'])


All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


joy 0.9986829161643982


In [3]:
import re
def nlp_pipeline(text):

    text = text.lower()
    text = text.replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"(\s\-\s|-$)", "", text)
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\&", "", text)
    text = re.sub(r"\+", "", text)
    text = re.sub(r"\#", "", text)
    text = re.sub(r"\$", "", text)
    text = re.sub(r"\£", "", text)
    text = re.sub(r"\%", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\@", "", text)
    text = re.sub(r"\-", "", text)

    return text
df["text_cleaned"]=df["intervention"].apply(lambda x: nlp_pipeline(x))

In [4]:
small_df = df.sample(100)

In [6]:
emotions = []
cnt = 0
for x in df["text_cleaned"]:
    if len(x.split(" ")) > 100:
        tmp = []
        for i in range(0, len(x.split(" ")), 100):
            tmp.append(emotion_classifier(" ".join(x.split(" ")[i:i+100]))[0])
        emotions.append(tmp)
    else :
        emotions.append(emotion_classifier(x))
    #print(cnt)
    cnt += 1

df["emotions"] = emotions

In [7]:
df.to_csv("tmp/interventions_emotions.csv", index=False)

Test not useful : 

In [41]:
from multiprocessing import Pool

def process_text(text):
    if len(text.split(" ")) > 100:
        tmp = []
        for i in range(0, len(text.split(" ")), 100):
            tmp.append(emotion_classifier(" ".join(text.split(" ")[i:i+100]))[0])
        return tmp
    else :
        return emotion_classifier(text)

def process_dataframe(df):
    with Pool() as pool:
        emotions = pool.map(process_text, df["text_cleaned"])
    df["emotions"] = emotions
    return df
process_dataframe(df)

In [38]:
import swifter

def split_array(arr):
    result = []
    for i in range(0, len(arr), 100):
        result.append(arr[i:i+100])
    return result

small_df["emotions"] = small_df["intervention"].apply(lambda x: 
    emotion_classifier(x) if len(x.split(" ")) < 100 
    else 
    [emotion_classifier(y) for y in split_array(x.split(" "))]
)

End of test

In [8]:
df.head()

,intervenant,parti,intervention,text_cleaned,emotions
0,Yaël Braun-Pivet,REN,\nL'ordre du jour appelle la discussion du pro...,l'ordre du jour appelle la discussion du proje...,"[{'label': 'joy', 'score': 0.635474443435669}]"
1,Danièle Obono,LFI,"\nNon, pas question !",non pas question,"[{'label': 'anger', 'score': 0.8457685112953186}]"
2,Louis Boyard,LFI,"\nNon, ça ne va pas se passer comme ça ! Vous ...",non ça ne va pas se passer comme ça vous n'av...,"[{'label': 'anger', 'score': 0.5996062755584717}]"
3,Yaël Braun-Pivet,REN,"\nLa parole est à M. le ministre du travail, d...",la parole est à m. le ministre du travail du p...,"[{'label': 'joy', 'score': 0.5436665415763855}]"
4,André Chassaigne,GDR,\nSuspension ! La suspension est de droit !,suspension la suspension est de droit,"[{'label': 'anger', 'score': 0.7486376166343689}]"


In [10]:
for x in df.iterrows():
    print(x[1]["emotions"])

[{'label': 'joy', 'score': 0.635474443435669}]
[{'label': 'anger', 'score': 0.8457685112953186}]
[{'label': 'anger', 'score': 0.5996062755584717}]
[{'label': 'joy', 'score': 0.5436665415763855}]
[{'label': 'anger', 'score': 0.7486376166343689}]
[{'label': 'anger', 'score': 0.9827337265014648}]
[{'label': 'joy', 'score': 0.5032527446746826}]
[{'label': 'anger', 'score': 0.8442186117172241}]
[{'label': 'fear', 'score': 0.47682079672813416}]
[{'label': 'anger', 'score': 0.5861321091651917}]
[{'label': 'anger', 'score': 0.5861321091651917}]
[{'label': 'anger', 'score': 0.5861321091651917}]
[{'label': 'anger', 'score': 0.5861321091651917}]
[{'label': 'anger', 'score': 0.7112295627593994}]
[{'label': 'joy', 'score': 0.5409066677093506}]
[{'label': 'anger', 'score': 0.5521247386932373}]
[{'label': 'anger', 'score': 0.6129369735717773}]
[{'label': 'anger', 'score': 0.7269113659858704}]
[{'label': 'joy', 'score': 0.5000565648078918}]
[{'label': 'anger', 'score': 0.5915036201477051}]
[{'label': 

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
from sumy.summarizers.edmundson import EdmundsonSummarizer

text = "Here is an example sentence. Another sentence is this one. This is the last sentence."
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
sentences = sent_tokenize(text)
words = [word_tokenize(sentence.lower()) for sentence in sentences]
filtered_words = [[stemmer.stem(word) for word in words if word not in stop_words] for words in words]

word_frequencies = FreqDist([word for sentence in filtered_words for word in sentence])

summarizer = EdmundsonSummarizer()
summarizer.bonus_words = filtered_words[0]
summarizer.stigma_words = filtered_words[1]
summarizer.null_words = filtered_words[2]

summary = summarizer(document, sentence_count=3)


In [ ]:
from gensim.summarization.summarizer import EdmundsonSummarizer
summarizer = EdmundsonSummarizer()

# set the positive and negative keywords and weights
summarizer.positive_keywords = ['good', 'great', 'excellent']
summarizer.negative_keywords = ['bad', 'poor', 'terrible']
summarizer.positive_weight = 1.5
summarizer.negative_weight = -1.5

# set the length of the summary you want to generate
summary_length = 3

In [ ]:
df.head()